
This notebook is used to test two different deep neural networks on one datasets



In [1]:
#install the necessary modules
!pip install torch
!pip install torchvision
!pip install matplotlib
!pip install scikit-learn

In [3]:
import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import CosineAnnealingLR
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import time


In [3]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import json
import time


In [4]:
# Load CIFAR-10 dataset
transform = transforms.Compose(
    [transforms.RandomHorizontalFlip(), 
     transforms.RandomCrop(32, padding=4), 
     transforms.ToTensor(), 
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

In [5]:
"""
The Cifar dataset is divided into two main folders: one for training and one for testing. To further improve the model's evaluation, 
I will create a validation set by extracting 5% of the training data. 
This validation set will allow me to assess the model's accuracy during training without using the test set. 
By validating on data the model has never encountered, I can better determine if the model is overfitting, ensuring a more generalizable performance.
        Validation set size: 1,000
        Test set size: 9,000
        Training set size: 50,000
"""
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Function to split the dataset
def dataset_split(test_dataset, val_ratio=0.1):
    # Calculate validation size based on the ratio
    val_size = int(len(test_dataset) * val_ratio)
    train_size = len(test_dataset) - val_size
    
    test_dataset, val_dataset = random_split(test_dataset, [train_size, val_size])
    
    val_loader = DataLoader(dataset=val_dataset, batch_size=64, shuffle=False)
    test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=True)
    
    return val_loader, test_loader

val_loader, test_loader = dataset_split(test_dataset, val_ratio=0.1)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=False)
print("Data loaded successfully")
print("Validation set size: ", len(val_loader.dataset))
print("Test set size: ", len(test_loader.dataset))
print("Training set size: ", len(train_loader.dataset))

Files already downloaded and verified
Files already downloaded and verified
Data loaded successfully
Validation set size:  1000
Test set size:  9000
Training set size:  50000


In [1]:
!pip install torchviz graphviz


In [6]:

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        # Layer 1: Conv2d + BatchNorm + ReLU + MaxPool
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5, padding=0),  # No padding
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # Max pool (28x28 -> 14x14)
        )
        
        # Layer 2: Conv2d + BatchNorm + ReLU + MaxPool
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=0),  # No padding
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # Max pool (14x14 -> 6x6)
        )
        
        # Fully connected layers
        self.fc1 = nn.Linear(64 * 6 * 6, 512)  # Input size 64 * 6 * 6 = 2304
        self.fc2 = nn.Linear(512, 10)  # Output size = 10 classes

    def forward(self, x):
        # Apply layer 1
        x = self.layer1(x)
        
        # Apply layer 2
        x = self.layer2(x)
        
        # Flatten the tensor
        x = x.view(x.size(0), -1)
        
        # Apply fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x



'cnn_model_architecture.dot'

[link text](https://)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.6)
scheduler = CosineAnnealingLR(optimizer, T_max=100, eta_min=0.001)


In [8]:
def train_and_evaluate(model, train_loader, test_loader,num_epochs): 
    # To store training data for plotting
    training_data = {
        'train_loss': [],
        'test_loss': [],
        'train_error': [],
        'test_error': [],
        'accuracy': [],
        'precision': [],
        'recall': [],
        'f1_score': []
    }

    start_time = time.time()

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        total_train_loss = 0
        correct_train = 0
        total_train = 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
        scheduler.step()
        avg_train_loss = total_train_loss / len(train_loader.dataset)
        train_error = 1 - (correct_train / total_train)

        # Testing phase
        model.eval()
        total_test_loss = 0
        correct_test = 0
        total_test = 0
        all_labels = []
        all_preds = []
        
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)
                total_test_loss += loss.item() * images.size(0)

                _, predicted = torch.max(outputs.data, 1)
                total_test += labels.size(0)
                correct_test += (predicted == labels).sum().item()

                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(predicted.cpu().numpy())

        avg_test_loss = total_test_loss / len(test_loader.dataset)
        test_error = 1 - (correct_test / total_test)

        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='weighted')
        recall = recall_score(all_labels, all_preds, average='weighted')
        f1 = f1_score(all_labels, all_preds, average='weighted')

        # Store metrics
        training_data['train_loss'].append(avg_train_loss)
        training_data['test_loss'].append(avg_test_loss)
        training_data['train_error'].append(train_error)
        training_data['test_error'].append(test_error)
        training_data['accuracy'].append(accuracy)
        training_data['precision'].append(precision)
        training_data['recall'].append(recall)
        training_data['f1_score'].append(f1)

        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Train Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}, '
              f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, '
              f'Recall: {recall:.4f}, F1 Score: {f1:.4f}')

    end_time = time.time()
    training_time = end_time - start_time
    training_data['training_time'] = training_time

    return training_data


In [9]:
def save_model_and_results(model, model_name, training_data):
    torch.save(model.state_dict(), f'{model_name}_model.pth')
    # Convert training data to a JSON-serializable format
    serializable_data = {}
    for key, value in training_data.items():
        if isinstance(value, torch.Tensor):
            serializable_data[key] = value.tolist()  # Convert Tensors to lists
        elif isinstance(value, list) and isinstance(value[0], torch.Tensor):
            serializable_data[key] = [v.tolist() for v in value]  # Convert lists of Tensors
        else:
            serializable_data[key] = value

    # Save the training data as JSON
    with open(f'{model_name}_training_data.json', 'w') as json_file:
        json.dump(serializable_data, json_file)


In [10]:
print(f'Training model : {model}')
training_data = train_and_evaluate(model, train_loader, test_loader, num_epochs=100)

save_model_and_results(model, f'DCNN', training_data)


Training model : CNNModel(
  (layer1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=2304, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)
Epoch [1/100], Train Loss: 1.5783, Test Loss: 1.5627, Accuracy: 0.4609, Precision: 0.5100, Recall: 0.4609, F1 Score: 0.4340
Epoch [2/100], Train Loss: 1.1827, Test Loss: 1.3597, Accuracy: 0.5416, Precision: 0.5706, Recall: 0.5416, F1 Score: 0.5246
Epoch [3/100], Train Loss: 1.0316, Test Loss: 1.

In [12]:
import torch

def evaluate_model(model, val_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# Loop through both models to evaluate them on the whole validation set
accuracy = evaluate_model(model, val_loader)
print(f'Accuracy for Network on the whole validation set: {accuracy:.2%}')


Accuracy for Network on the whole validation set: 81.70%
